In [1]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tqdm import trange
from time import sleep
use_gpu = torch.cuda.is_available()

###### ResNet Architecture

In [2]:

def conv3x3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3x3 convolution with padding"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1x1(in_planes, out_planes, stride=1):
    """1x1x1 convolution"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
    
    def forward(self, x):
        residual = x
        #print(x.shape)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        #print("basicblock1")
        #print(out.shape)
        
        
        out = self.conv2(out)
        out = self.bn2(out)
        #print("basicblock2")
        #print(out.shape)
        
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu2(out)
        #print("basicblock3")
        #print(out.shape)
        
        return out

class ResNet_3d(nn.Module):

    def __init__(self, n_seqs, block, layers, num_classes=2):
        
        self.inplanes = 16
        super(ResNet_3d, self).__init__()
        self.conv1 = nn.Conv3d(n_seqs, 16, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm3d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2)
        self.avgpool = nn.AvgPool3d(5, stride=1)
        self.fc = nn.Linear(128 * block.expansion, num_classes)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    
    def forward(self, x):
        x = self.conv1(x)    
        x = self.bn1(x)     
        x = self.relu(x)   # 
        #print("layer1")
        #print(x.shape)
        x = self.layer1(x)   
        #print("layer2")
        #print(x.shape)
        x = self.layer2(x)   
        #print("layer3")
        #print(x.shape)
        x = self.layer3(x)   
        #print("layer4")
        #print(x.shape)
        x = self.layer4(x)   
        #print("avgpool")
        #print(x.shape)
        
        x = self.avgpool(x)  
        #print("fc")
        #print(x.shape)
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = self.fc(x)

        return x


In [4]:
torch.cuda.device_count()
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [ ]:
model_abPET = ResNet_3d(1, BasicBlock, [3,4,6,3])
model_abPET_filename = './revisedResNet3d_abPET_72'
checkpoint = torch.load(model_abPET_filename)
model_abPET.load_state_dict(checkpoint['model_state_dict'])
model_abPET.eval()
model_abPET.cuda()